[this doc on github](https://github.com/dotnet/interactive/tree/master/samples/notebooks/fsharp/Docs)

# Displaying Output in an F# notebook

This article describes how to display output results in F# notebooks.

When writing F# in a .NET notebook, you can write code similar to how you would with the F# Interactive tool.

In [ ]:
1 + 3

The last value in a cell has its output displayed. When you end a cell with an expression evaluation like this, it is the return value of the cell. There can only be a single return value for a cell. If you add more code after a return value expression, you'll see a compile error. 

Here is another example:

In [ ]:
let r = System.Random()
r.Next(0,10)

You can also evaluate a string:

In [ ]:
"Hello, world!"


### Using Console Output

You can display information without using the return value. The most intuitive way is to write to the console:

In [ ]:
System.Console.WriteLine("Hello, world!")
printfn "Hello, world!"

You can also use `%A` printing to print structured values:

In [ ]:
printfn "%120A" [ for i in 1 .. 30 -> [ 1 .. i ]]

Console output produced after the completion of the cell execution (e.g. from background threads or asynchronous processing) may be ignored.

## Using `display`

A more familiar API for many notebook users would be the `display` method.

In [ ]:
display("Hello, world!")

Many outputs are automatically formatted as HTML by `display` or output-value formatting.



Each call to `display` writes an additional display data value to the notebook.
You can also update an existing displayed value by calling `Update` on the object returned by a call to `display`.

In some environments such as Jupyter Lab, only updates made during the execution of the cell are applied.  Any updates made by background processing after the completion of
execution may be ignored.

In [ ]:
let fruitOutput = display("Let's get some fruit!");
let basket = [| "apple"; "orange"; "coconut"; "pear"; "peach" |]

for fruit in basket do
    System.Threading.Thread.Sleep(1000);
    let updateMessage = sprintf "I have 1 %s" fruit
    fruitOutput.Update(updateMessage)

System.Threading.Thread.Sleep(1000);

fruitOutput.Update(basket);

## Displaying HTML



To display arbitrary HTML, the helper function `HTML` can be used.


In [ ]:
HTML(Html.raw "<b style=\"color:blue\">Hello!</b>")

HTML can be specified using any DSL that generates ASP.NET Core `IHtmlContent`.  The `Html` module contains an F# DSL for HTML specification similar to [Giraffe.ViewEngine](https://github.com/giraffe-fsharp/Giraffe.ViewEngine#html-elements-and-attributes) that does this. For example:


In [ ]:
open Html

let indexView =
    div [] [
      h4 [] [ str "I love F#" ]
      p [ _class "some-css-class"; _id "someId" ] [
          b  [] [str "Hello" ]
          str " "
          i [] [str "World"]
      ]
    ]

display(indexView)

## Executing Javascript

The helper function `Javascript` executes Javascript directly.

In [ ]:
Javascript(@"alert(""Hello!"");")

## Defining CSS

The `CSS` function can be used to add CSS styling to the host HTML system. Here are some examples:

In [ ]:
CSS "h2 { color: darkslategrey; }"